In [26]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg, FQ, final_exponentiate, FQ12
import galois
import numpy as np

# p = 71
# p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
FP = galois.GF(curve_order)

x = FP(5)
y = FP(10)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + FP(2) * v4 - FP(5) * x * v3 - FP(3) * y + FP(2)

w = FP([1, out, x, y, v1, v2, v3, v4])

print("w =", w)

L = FP([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,curve_order - 5,0,0,0,0,0] # 113 - 5 = 108
    ])

R = FP(
    [
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]
)

O = FP([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [FP(curve_order-2),1,0,3,0,FP(curve_order-1),0,FP(curve_order-2)] # 113 - 2 = 111
    ])

Lw = np.dot(L, w)
Rw = np.dot(R, w)
Ow = np.dot(O, w)

print("Lw =", Lw)
print("Rw =", Rw)

LwRw = np.multiply(Lw, Rw)

print("Lw * Rw =", LwRw)

print("Ow =     ", Ow)

assert np.all(LwRw == Ow)


w = [   1   97    5   10   25  125  100 1250]
Lw = [                                                                            5
                                                                            25
                                                                            10
                                                                           125
 21888242871839275222246405745257275088548364400416034343698204186575808495592]
Rw = [  5   5  10  10 100]
Lw * Rw = [                                                                           25
                                                                           125
                                                                           100
                                                                          1250
 21888242871839275222246405745257275088548364400416034343698204186575808493117]
Ow =      [                                                                           25
                               

In [27]:
# QAP
mtxs = [L, R, O]
poly_m = []

for m in mtxs:
    poly_list = []
    for i in range(0, m.shape[1]):
        points_x = FP(np.zeros(m.shape[0], dtype=int))
        points_y = FP(np.zeros(m.shape[0], dtype=int))
        for j in range(0, m.shape[0]):
            points_x[j] = FP(j+1)
            points_y[j] = m[j][i]

        poly = galois.lagrange_poly(points_x, points_y)
        coef = poly.coefficients()[::-1]
        if len(coef) < m.shape[0]:
            coef = np.append(coef, np.zeros(m.shape[0] - len(coef), dtype=int))
        poly_list.append(coef)
    
    poly_m.append(FP(poly_list))

Lp = poly_m[0]
Rp = poly_m[1]
Op = poly_m[2]

print(f'''L
{Lp}
''')

print(f'''R
{Rp}
''')

print(f'''O
{Op}
''')

L
[[                                                                            0
                                                                              0
                                                                              0
                                                                              0
                                                                              0]
 [                                                                            0
                                                                              0
                                                                              0
                                                                              0
                                                                              0]
 [                                                                            0
                                                                              4
   729608095727975840741546858175242

In [43]:
print("Setup phase")
print("-"*10)
print("Toxic waste:")
tau = FP(20)

print(f"τ = {tau}")

T = galois.Poly([1, curve_order-1], field=FP)
for i in range(2, L.shape[0] + 1):
    T *= galois.Poly([1, curve_order-i], field=FP)

print("\nT = ", T)
for i in range(1, L.shape[0] + 2):
    print(f"T({i}) = ", T(i))
    if i == L.shape[0]:
        print("-"*10)

T_tau = T(tau)
print(f"\nT(τ) = {T_tau}")

Setup phase
----------
Toxic waste:
τ = 20

T =  x^5 + 21888242871839275222246405745257275088548364400416034343698204186575808495602x^4 + 85x^3 + 21888242871839275222246405745257275088548364400416034343698204186575808495392x^2 + 274x + 21888242871839275222246405745257275088548364400416034343698204186575808495497
T(1) =  0
T(2) =  0
T(3) =  0
T(4) =  0
T(5) =  0
----------
T(6) =  120

T(τ) = 1395360


In [29]:
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize

# G1[τ^0], G1[τ^1], ..., G1[τ^d]
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
# G1[τ^0 * T(τ)], G1[τ^1 * T(τ)], ..., G1[τ^d-1 * T(τ)]
target_G1 = [multiply(G1, int(tau**i * T_tau)) for i in range(0, T.degree - 1)]

# G2[τ^0], G2[τ^1], ..., G2[τ^d-1]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]

print("Trusted setup:")
print("-"*10)
print(f"[τ]G1 = {[point for point in tau_G1]}")
print(f"[T(τ)]G1 = {[point for point in target_G1]}")

print(f"\n[τ]G2 = {[point for point in tau_G2]}")

Trusted setup:
----------
[τ]G1 = [(1, 2, 1), (3495472558767638062130276051692996261059606098942701143640544070940431621895, 21818578158853851286081718859129094794224735409505575270300354953849943283096, 13539030627541719173126364779926882740848856514394536490700044749360163567065), (2930071262840081682787806351141638544542011873723835099714634535544099644269, 20922112042347544862375321327802563994838974782520985075729130372912875807010, 5188714476106825629983626619212640312144182290814903660574387364421666278298), (12538660806407328503005847646722268159776993443150219500574976971817785295217, 5727651509046529781720451554549025178698161406046510794293584612214748669439, 18762028699045338833195414421096472129350075620097676745236597945317958322084), (7260371114671341159978105380960553597729240885090899052071662815546928639239, 17042378484016954407363867564165615851260867840405348772808465166103895929461, 19524649339789898409678197077959127869439524671820525064660413955319287324933)]
[T(

In [42]:
U = galois.Poly((w @ Lp)[::-1])
V = galois.Poly((w @ Rp)[::-1])
W = galois.Poly((w @ Op)[::-1])
print("TTTTTTT",T)
# print("HHHHHHH",H)


print("U = ", U)
print("V = ", V)
print("W = ", W)

H = (U * V - W) // T
rem = (U * V - W) % T

print("H = ", H)
print("rem = ", rem)

assert rem == 0

u = U(tau)
v = V(tau)
_w = W(tau)
ht = H(tau)*T_tau

print("u",u)
print("v",v)
print("_w",_w)
print("ht",ht)

assert u * v - _w == ht, f"{u} * {v} - {_w} != {ht}"

TTTTTTT x^5 + 21888242871839275222246405745257275088548364400416034343698204186575808495602x^4 + 85x^3 + 21888242871839275222246405745257275088548364400416034343698204186575808495392x^2 + 274x + 21888242871839275222246405745257275088548364400416034343698204186575808495497
HHHHHHH 1824020239319939601853867145438106257379030366701336195308183682214650707866x^3 + 8360092763549723175163557749924653679653889180714457561829175210150482412190x^2 + 16720185527099446350327115499849307359307778361428915123658350420300964821680x + 14592161914559516814830937163504850059032242933610689562465469457717205664478
U =  7296080957279758407415468581752425029516121466805344781232734728858602831849x^4 + 3648040478639879203707734290876212514758060733402672390616367364429301416197x^3 + 3648040478639879203707734290876212514758060733402672390616367364429301414937x^2 + 7296080957279758407415468581752425029516121466805344781232734728858602833414x + 21888242871839275222246405745257275088548364400416034343698204186

In [45]:
def evaluate_poly(poly, trusted_points, verbose=False):
    coeff = poly.coefficients()[::-1]

    assert len(coeff) == len(trusted_points), "Polynomial degree mismatch!"

    if verbose:
        [print(point) for point in trusted_points]

    terms = [multiply(point, int(coeff)) for point, coeff in zip(trusted_points, coeff)]
    evaluation = terms[0]
    for i in range(1, len(terms)):
        evaluation = add(evaluation, terms[i])

    if verbose:
        print("-"*10)
        print(evaluation)
    return evaluation

print("\nProof generation:")
print("-"*10)
# G1[u0 * τ^0] + G1[u1 * τ^1] + ... + G1[ud-1 * τ^d-1]
A_G1 = evaluate_poly(U, tau_G1)
# G2[v0 * τ^0] + G2[v1 * τ^1] + ... + G2[vd-1 * τ^d-1]
B_G2 = evaluate_poly(V, tau_G2)
# G1[w0 * τ^0] + G1[w1 * τ^1] + ... + G1[wd-1 * τ^d-1]
B_G1 = evaluate_poly(V, tau_G1)
# G1[w0 * τ^0] + G1[w1 * τ^1] + ... + G1[wd-1 * τ^d-1]
Cw_G1 = evaluate_poly(W, tau_G1)
# G1[h0 * τ^0 * T(τ)] + G1[h1 * τ^1 * T(τ)] + ... + G1[hd-2 * τ^d-2 * T(τ)]
HT_G1 = evaluate_poly(H, target_G1)

C_G1 = add(Cw_G1, HT_G1)

print(f"[A]G1 = {A_G1}")
print(f"[B]G2 = {B_G2}")
print(f"[C]Cw = {Cw_G1}")
print(f"[C]HTG1 = {HT_G1}")
print(f"[C]G1 = {C_G1}")

print(A_G1)

print("\nProof verification:")
print("-"*10)
# e(A, B) == e(C, G2[1])
assert pairing(B_G2, A_G1) == pairing(G2, C_G1), "Pairing check failed!"
print("Pairing check passed!")


Proof generation:
----------
[A]G1 = (10761961764129973982054763719362943500866170651873734401430882177283055329417, 5058895679614798810943118395621508047500638844754741455340774317600459389282)
[B]G2 = ((14676596619546708998128589777444655676977278549271023549538909836586459894341, 11919835763118668667740900971595365782274213478929042926599835422246695122845), (4181410176679702185692223732593891253384989642736027109804322446705565922423, 8269324330636396465404020993058672731247265150570478427587813989536384471923))
[C]Cw = (11006377201644525345426928991404003103659940521647545481641368342770471795148, 5625735634426074623513914315028312426791663040205825753846553583577855352457)
[C]HTG1 = (7590285867717173171246348344240436388940237951733020390870567473915245420644, 16394849742817848566784053762791793238022916377975074470311212431696897590925)
[C]G1 = (10090448916825695921462497574579247490164896334824172194227448917125407270189, 3548740209221560558115541547059690110665779583138513567

In [47]:
alpha = FP(2)
beta = FP(3)

alpha_G1 = multiply(G1, int(alpha))
beta_G2 = multiply(G2, int(beta))

A_G1_alpha = add(A_G1, alpha_G1)
B_G2_beta = add(B_G2, beta_G2)


# AB = pairing(B_G2, A_G1)
# CD = pairing(G2, C_G1)
# EF = pairing(beta_G2, alpha_G1)
# eq(AB, CD * EF)
# # final_exponentiate(AB * CD * EF) == FQ12.one()


In [56]:
def evaluate_poly_list(poly_list, x):
    results = []
    for poly in poly_list:
        results.append(poly(x))
    return results

def print_evaluation(name, results):
    print(f'\n{name} polynomial evaluations:')
    for i in range(0, len(results)):
        print(f'{name}_{i} = {results[i]}')

def to_poly(mtx):
    poly_list = []
    for i in range(0, mtx.shape[0]):
        poly_list.append( galois.Poly(mtx[i][::-1]) )
    return poly_list

def print_poly(name, poly_list):
    print(f'\n{name} polynomials:')
    for i in range(0, len(poly_list)):
        print(f'{name}_{i} = {poly_list[i]}')

beta_L = beta * Lp
alpha_R = alpha * Rp

# print("βL\n", beta_L)
# print("αR\n", alpha_R)
# print("O\n", Op)

K = beta_L + alpha_R + Op 
print("K = βL + αR + O\n", K)

Kp = to_poly(K)
print_poly("K", Kp)

print("K evaluations:")
K_eval = evaluate_poly_list(Kp, tau)
print([int(k) for k in K_eval])

K = βL + αR + O
 [[21888242871839275222246405745257275088548364400416034343698204186575808495615
  18240202393199396018538671454381062573790303667013361953081836822146507079685
  20064222632519335620392538599819168831169334033714698148390020504361157787646
   3648040478639879203707734290876212514758060733402672390616367364429301415937
   1824020239319939601853867145438106257379030366701336195308183682214650707968]
 [                                                                            1
   1824020239319939601853867145438106257379030366701336195308183682214650707966
  11856131555579607412050136445347690672963697383558685269503193934395229601794
   9120101196599698009269335727190531286895151833506680976540918411073253539840
  20976232752179305421319472172538221959858849217065366246044112345468483141633]
 [21888242871839275222246405745257275088548364400416034343698204186575808495607
   3648040478639879203707734290876212514758060733402672390616367364429301415971
  1641618215387945641

In [58]:
from py_ecc.bn128 import multiply, G1, G2, add, pairing, neg, eq

# G1[α]
alpha_G1 = multiply(G1, int(alpha))
# G1[β]
beta_G1 = multiply(G1, int(beta))
# G1[τ^0], G1[τ^1], ..., G1[τ^d]
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
# G1[βU0(τ) + αV0(τ) + W0(τ)], G1[βU1(τ) + αV1(τ) + W1(τ)], ..., G1[βUd(τ) + αVd(τ) + Wd(τ)]
k_G1 = [multiply(G1, int(k)) for k in K_eval]
print(f"k_G1 = {k_G1}")
# G1[τ^0 * T(τ)], G1[τ^1 * T(τ)], ..., G1[τ^d-1 * T(τ)]
target_G1 = [multiply(G1, int(tau**i * T_tau)) for i in range(0, T.degree - 1)]

# G2[α]
beta_G2 = multiply(G2, int(beta))
# G2[τ^0], G2[τ^1], ..., G2[τ^d-1]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]

print("Trusted setup:")
print("-"*10)
print(f"[α]G1 = {alpha_G1}")
print(f"[β]G1 = {beta_G1}")
print(f"[τ]G1 = {[point for point in tau_G1]}")
print(f"[k]G1 = {[point for point in k_G1]}")
print(f"[τT(τ)]G1 = {[point for point in target_G1]}")

print(f"\n[β]G2 = {beta_G2}")
print(f"[τ]G2 = {[point for point in tau_G2]}")

k_G1 = [(18412086319910179752494782965811456442455794258224578119780579512176567504706, 10558562959246852549340863372112227129585961702953419993535490858069696891127), (17518925876389381914155930110408777142106130111113028150026931997812097774535, 8858392472586588859720780881006689049064823023738566495968587324881339689331), (9768342630219773912066130367574825758751376420955732609545234776699038812204, 10720364412465394116201770343713162737457451428979397953573675794735466940134), (17220357501765311014989932181533273340703745296812138843141373296717590992453, 12911075614676998844858506464301523892884475566951707630884705350159675739690), (13773034646707703060024638388908327476085926589455715888797088238131068849198, 6652088763425185776595483932916192818167293982963912677776663936238090319287), (18868485492852322166852649924742914652689950572147954212651748232358386529165, 10103508926598196183095849357157716596646267673423991925597149801350629183052), (1032465337081544412436086770829983

In [53]:
print("\nProof generation:")
print("-"*10)
# G1[u0 * τ^0] + G1[u1 * τ^1] + ... + G1[ud-1 * τ^d-1]
A_G1 = evaluate_poly(U, tau_G1)
# G1[A] = G1[A] + G1[α]
A_G1 = add(A_G1, alpha_G1)
# G2[v0 * τ^0] + G2[v1 * τ^1] + ... + G2[vd-1 * τ^d-1]
B_G2 = evaluate_poly(V, tau_G2)
# G2[B] = G2[B] + G2[β]
B_G2 = add(B_G2, beta_G2)
# G1[h0 * τ^0 * T(τ)] + G1[h1 * τ^1 * T(τ)] + ... + G1[hd-2 * τ^d-2 * T(τ)]
HT_G1 = evaluate_poly(H, target_G1)
# w0 * G1[k0] + w1 * G1[k1] + ... + wd-1 * G1[kd-1]
assert len(w) == len(k_G1), "Polynomial degree mismatch!"
K_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(k_G1, w)]
K_G1 = K_G1_terms[0]
for i in range(1, len(K_G1_terms)):
    K_G1 = add(K_G1, K_G1_terms[i])

C_G1 = add(HT_G1, K_G1)


print(f"HTG1 = {HT_G1}")
print(f"K_G1 = {K_G1}")
print(f"")
print(f"[A]G1 = {A_G1}")
print(f"[B]G2 = {B_G2}")
print(f"[C]G1 = {C_G1}")
print("-" * 10)
print("Verifier computes:")
print(f"[α]G1 = {alpha_G1}")
print(f"[β]G1 = {beta_G1}")

lhs = pairing(B_G2, A_G1)
rhs = pairing(G2, C_G1) * pairing(beta_G2, alpha_G1)

print(eq(lhs,rhs))


Proof generation:
----------
HTG1 = (7590285867717173171246348344240436388940237951733020390870567473915245420644, 16394849742817848566784053762791793238022916377975074470311212431696897590925)
K_G1 = (18086161042831484437962532820846621558004793007225895387073937607441315584160, 7045536001405948878446706149289882186407672609178012768003543711528611060931)

[A]G1 = (4765334342109574250419466061216441003374635556896557192186718797080873032790, 14842632034678405693773926112198003927403343823916552583863654295664758622551)
[B]G2 = ((2636638015587609787902059264018547527301759018048299575502007612191841858915, 14910301910690498693277239534140641274306409108383028639467876474455258217179), (9563240213990629511533451222988481310425270212120849282465248629834788569301, 17819757907105215667270026571474554629477867810720975049726959757254203443717))
[C]G1 = (13244387753095983615690676983597337060509513828536840239202676147687064283580, 9537609461377328351341973830666070143748190538554377741901

In [59]:
# DEBUG

def split_poly(poly):
    coef = [int(c) for c in poly.coefficients()]
    p1 = coef[-2:]
    p2 = coef[:-2] + [0] * 2

    return galois.Poly(p1, field=FP), galois.Poly(p2, field=FP)

u = U(tau)
v = V(tau)
_w = W(tau)
ht = H(tau)*T_tau

assert u * v == _w + ht

U1, U2 = split_poly(U)
V1, V2 = split_poly(V)
W1, W2 = split_poly(W)

w1 = W1(tau)
w2 = W2(tau)

u1 = U1(tau)
u2 = U2(tau)

v1 = V1(tau)
v2 = V2(tau)

c = (beta * u2 + alpha * v2 + w2) + ht 
k = (beta * u1 + alpha * v1 + w1)

assert (u + alpha) * (v + beta) == alpha * beta + k + c

In [61]:
k_pub_G1, k_priv_G1 = k_G1[:2], k_G1[2:]
pub_input, priv_input = w[:2], w[2:]

print(f"[k_pub]G1 = {[point for point in k_pub_G1]}")
print(f"[k_priv]G1 = {[point for point in k_priv_G1]}")

print(f"pub_input = {pub_input}")
print(f"priv_input = {priv_input}")

[k_pub]G1 = [(18412086319910179752494782965811456442455794258224578119780579512176567504706, 10558562959246852549340863372112227129585961702953419993535490858069696891127), (17518925876389381914155930110408777142106130111113028150026931997812097774535, 8858392472586588859720780881006689049064823023738566495968587324881339689331)]
[k_priv]G1 = [(9768342630219773912066130367574825758751376420955732609545234776699038812204, 10720364412465394116201770343713162737457451428979397953573675794735466940134), (17220357501765311014989932181533273340703745296812138843141373296717590992453, 12911075614676998844858506464301523892884475566951707630884705350159675739690), (13773034646707703060024638388908327476085926589455715888797088238131068849198, 6652088763425185776595483932916192818167293982963912677776663936238090319287), (18868485492852322166852649924742914652689950572147954212651748232358386529165, 10103508926598196183095849357157716596646267673423991925597149801350629183052), (103246533708154

In [62]:
K_pub_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(k_pub_G1, pub_input)]
K_pub_G1 = K_pub_G1_terms[0]
for i in range(1, len(K_pub_G1_terms)):
    K_pub_G1 = add(K_pub_G1, K_pub_G1_terms[i])

print(f"[K_pub]G1 = {K_pub_G1}")

K_priv_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(k_priv_G1, priv_input)]
K_priv_G1 = K_priv_G1_terms[0]
for i in range(1, len(K_priv_G1_terms)):
    K_priv_G1 = add(K_priv_G1, K_priv_G1_terms[i])

print(f"[K_priv]G1 = {K_priv_G1}")

[K_pub]G1 = (17006384426716725594429222955901978173864520248629640584450351743895359567248, 12935903412021047033591017961454890802467379243446965989690889808085263700897)
[K_priv]G1 = (21629268750396753292511157911471849797932765012719660569653001106721570967185, 5572122204216364758020501871637377523783536273959062440567939786399526980779)


In [63]:
C_G1 = add(HT_G1, K_priv_G1)

print("\nProof generation:")
print("-"*10)
print(f"[A]G1 = {A_G1}")
print(f"[B]G2 = {B_G2}")
print(f"[C]G1 = {C_G1}")
print("-" * 10)
print("Verifier uses:")
print(f"[α]G1 = {alpha_G1}")
print(f"[β]G1 = {beta_G1}")
print(f"[K terms]G1 = {[p for p in k_pub_G1]}")


Proof generation:
----------
[A]G1 = (4765334342109574250419466061216441003374635556896557192186718797080873032790, 14842632034678405693773926112198003927403343823916552583863654295664758622551)
[B]G2 = ((2636638015587609787902059264018547527301759018048299575502007612191841858915, 14910301910690498693277239534140641274306409108383028639467876474455258217179), (9563240213990629511533451222988481310425270212120849282465248629834788569301, 17819757907105215667270026571474554629477867810720975049726959757254203443717))
[C]G1 = (17952456226604690095434165940647630151850892666291761650384158825025480657570, 18956682402337804644191475827930836205265350967444861947787472168774214085567)
----------
Verifier uses:
[α]G1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
[β]G1 = (3353031288059533942658390886683067124040920775575537747144343083137631628272, 19321533766552368860946552437480

In [67]:
print("Setup phase")
print("-"*10)
print("Toxic waste:")
alpha = FP(2)
beta = FP(3)
gamma = FP(4)
delta = FP(5)
tau = FP(20)

print(f"α = {alpha}")
print(f"β = {beta}")
print(f"γ = {gamma}")
print(f"δ = {delta}")
print(f"τ = {tau}")

Setup phase
----------
Toxic waste:
α = 2
β = 3
γ = 4
δ = 5
τ = 20


In [68]:
def split_poly(poly):
    coef = [int(c) for c in poly.coefficients()]
    p1 = coef[-2:]
    p2 = coef[:-2] + [0] * 2

    return galois.Poly(p1, field=FP), galois.Poly(p2, field=FP)

# U, V, W - witness already computed

u = U(tau)
v = V(tau)
_w = W(tau)
ht = H(tau)*T_tau

assert u * v == _w + ht

U1, U2 = split_poly(U)
V1, V2 = split_poly(V)
W1, W2 = split_poly(W)

w1 = W1(tau)
w2 = W2(tau)

u1 = U1(tau)
u2 = U2(tau)

v1 = V1(tau)
v2 = V2(tau)

c = (beta * u2 + alpha * v2 + w2) * delta**-1 + ht * delta**-1
k = (beta * u1 + alpha * v1 + w1) * gamma**-1

a = u + alpha
b = v + beta

assert a * b == alpha * beta + k * gamma + c * delta

alpha_G1 = multiply(G1, int(alpha))
beta_G2 = multiply(G2, int(beta))
gamma_G2 = multiply(G2, int(gamma))
delta_G2 = multiply(G2, int(delta))

In [70]:
tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, T.degree)]
tau_G2 = [multiply(G2, int(tau**i)) for i in range(0, T.degree)]

powers_tauTtau_div_delta = [(tau**i * T_tau) / delta for i in range(0, T.degree - 1)]
target_G1 = [multiply(G1, int(pTd)) for pTd in powers_tauTtau_div_delta]

assert len(target_G1) == len(H.coefficients()), f"target_G1 length mismatch! {len(target_G1)} != {len(H.coefficients())}"

print("Trusted setup:")
print("-"*10)
print(f"[α]G1 = {alpha_G1}")
print(f"[β]G2 = {beta_G2}")
print(f"[γ]G2 = {gamma_G2}")
print(f"[δ]G2 = {delta_G2}")
print(f"[τ]G1 = {[point for point in tau_G1]}")
print(f"[τ]G2 = {[point for point in tau_G2]}")
print(f"[τT(τ)/δ]G1 = {[point for point in target_G1]}")

Trusted setup:
----------
[α]G1 = (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
[β]G2 = ((2725019753478801796453339367788033689375851816420509565303521482350756874229, 7273165102799931111715871471550377909735733521218303035754523677688038059653), (2512659008974376214222774206987427162027254181373325676825515531566330959255, 957874124722006818841961785324909313781880061366718538693995380805373202866))
[γ]G2 = ((18936818173480011669507163011118288089468827259971823710084038754632518263340, 18556147586753789634670778212244811446448229326945855846642767021074501673839), (18825831177813899069786213865729385895767511805925522466244528695074736584695, 13775476761357503446238925910346030822904460488609979964814810757616608848118))
[δ]G2 = ((20954117799226682825035885491234530437475518021362091509513177301640194298072, 4540444681147253467785307942530223364530218361853237193970751657229

In [71]:
w_pub = w[:2]
w_priv = w[2:]

K_gamma, K_delta = [k/gamma for k in K_eval[:2]], [k/delta for k in K_eval[2:]]

print(f"K/γ = {[int(k) for k in K_gamma]}")
print(f"K/δ = {[int(k) for k in K_delta]}")

K_gamma_G1 = [multiply(G1, int(k)) for k in K_gamma]
K_delta_G1 = [multiply(G1, int(k)) for k in K_delta]

print(f"[K/γ]G1 = {[point for point in K_gamma_G1]}")
print(f"[K/δ]G1 = {[point for point in K_delta_G1]}")

# [K/γ*w]G1
Kw_gamma_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(K_gamma_G1, w_pub)]
Kw_gamma_G1 = Kw_gamma_G1_terms[0]
for i in range(1, len(Kw_gamma_G1_terms)):
    Kw_gamma_G1 = add(Kw_gamma_G1, Kw_gamma_G1_terms[i])

print(f"[K/γ*w]G1 = {Kw_gamma_G1}")

# [K/δ*w]G1
Kw_delta_G1_terms = [multiply(point, int(scaler)) for point, scaler in zip(K_delta_G1, w_priv)]
Kw_delta_G1 = Kw_delta_G1_terms[0]
for i in range(1, len(Kw_delta_G1_terms)):
    Kw_delta_G1 = add(Kw_delta_G1, Kw_delta_G1_terms[i])

print(f"[K/δ*w]G1 = {Kw_delta_G1}")

K/γ = [21888242871839275222246405745257275088548364400416034343698204186575808493679, 969]
K/δ = [21888242871839275222246405745257275088548364400416034343698204186575808481881, 4377648574367855044449281149051455017709672880083206868739640837315161716155, 21888242871839275222246405745257275088548364400416034343698204186575808488477, 13132945723103565133347843447154365053129018640249620606218922511945485085290, 17510594297471420177797124596205820070838691520332827474958563349260646802148, 4377648574367855044449281149051455017709672880083206868739640837315161694666]
[K/γ]G1 = [(15470831961688208669697575643418221048181962981001546150468987941769939214029, 18777991423107314776482652904700650180890186299459272499931369322486171899854), (1540147615742092855122362390273215696202300989980535937807544369063691019287, 916302540883947080468652904864832717784863855769927076757632797189693955321)]
[K/δ]G1 = [(16524110854952697577386273560426967717495967530452967658401010590431306966138, 16823085444

In [72]:
HT_G1 = evaluate_poly(H, target_G1)
C_G1 = add(Kw_delta_G1, HT_G1)
K_G1 = Kw_gamma_G1

A_G1 = evaluate_poly(U, tau_G1)
A_G1 = add(A_G1, alpha_G1)
B_G2 = evaluate_poly(V, tau_G2)
B_G2 = add(B_G2, beta_G2)

k1 = K_gamma_G1[0]
k2 = K_gamma_G1[1]

In [73]:
HT_G1 = evaluate_poly(H, target_G1)
print(f"\n[τT(τ)/δ]G1 = {HT_G1}")

assert pairing(G2, multiply(G1, int(ht/delta))) == pairing(G2, HT_G1)


[τT(τ)/δ]G1 = (18876847322725696765527933193224150241343049669725842846807522920161435499280, 13368826749593275882644022789382492396451241563856671889711878741729795394019)


In [74]:
print("Additional Trusted setup:")
print("-"*10)
delta_G1 = multiply(G1, int(delta))
gamma_G1 = multiply(G1, int(gamma))

print(f"[γ]G1 = {gamma_G1}")
print(f"[δ]G1 = {delta_G1}")


Additional Trusted setup:
----------
[γ]G1 = (3010198690406615200373504922352659861758983907867017329644089018310584441462, 4027184618003122424972590350825261965929648733675738730716654005365300998076)
[δ]G1 = (10744596414106452074759370245733544594153395043370666422502510773307029471145, 848677436511517736191562425154572367705380862894644942948681172815252343932)


In [76]:
print("Prover picks random r, s")
r = FP(6)
s = FP(8)

print(f"r = {r}")
print(f"s = {s}")

def split_poly(poly):
    coef = [int(c) for c in poly.coefficients()]
    p1 = coef[-2:]
    p2 = coef[:-2] + [0] * 2

    return galois.Poly(p1, field=FP), galois.Poly(p2, field=FP)

# U, V, W - witness already computed
u = U(tau)
v = V(tau)
_w = W(tau)
ht = H(tau)*T_tau

assert u * v == _w + ht

U1, U2 = split_poly(U)
V1, V2 = split_poly(V)
W1, W2 = split_poly(W)

w1 = W1(tau)
w2 = W2(tau)

u1 = U1(tau)
u2 = U2(tau)

v1 = V1(tau)
v2 = V2(tau)

a = u + alpha + r * delta
b = v + beta + s * delta

c = ((beta * u2 + alpha * v2 + w2) * delta**-1 + ht * delta**-1) + s * a + r * b - r * s * delta
k = (beta * u1 + alpha * v1 + w1) * gamma**-1

assert a * b == alpha * beta + k * gamma + c * delta


Prover picks random r, s
r = 6
s = 8


In [ ]:
r_delta_G1 = multiply(delta_G1, int(r))
s_delta_G1 = multiply(delta_G1, int(s))
s_delta_G2 = multiply(delta_G2, int(s))

A_G1 = evaluate_poly(U, tau_G1)
A_G1 = add(A_G1, alpha_G1)
A_G1 = add(A_G1, r_delta_G1)

B_G2 = evaluate_poly(V, tau_G2)
B_G2 = add(B_G2, beta_G2)
B_G2 = add(B_G2, s_delta_G2)

B_G1 = evaluate_poly(V, tau_G1)
B_G1 = add(B_G1, beta_G1)
B_G1 = add(B_G1, s_delta_G1)

As_G1 = multiply(A_G1, int(s))
Br_G1 = multiply(B_G1, int(r))
rs_delta_G1 = multiply(delta_G1, int(-r*s))

C_G1 = add(Kw_delta_G1, HT_G1)
C_G1 = add(C_G1, As_G1)
C_G1 = add(C_G1, Br_G1)
C_G1 = add(C_G1, rs_delta_G1)

print(f"[A]G1 = {A_G1}")
print(f"[B]G2 = {B_G2}")
print(f"[C]G1 = {C_G1}")